# N状態モデル
## 自己回帰確率過程の離散近似

- 前節では、マルコフ連鎖の取り得る値$s$や遷移確率$p_{ij}$は既に与えられていた

- Tauchen(1986)は、連続的(continuous)な値をとる、１階の自己回帰(first-order auto-regressive, AR(1))確率過程を、マルコフ連鎖で離散的(discrete)に近似する方法を考案した

- 変数$x\in\mathbb{R}$が、以下のようなAR(1)過程に従うとする

$$
  x' = c + \rho x + \varepsilon', \quad \varepsilon' \sim N(0,\sigma_{\varepsilon}^{2}) 
$$

$c$は定数、$\rho$は慣性についてのパラメータ、$\varepsilon'$は平均$0$、分散$\sigma_{\varepsilon}^{2}$の正規分布に従う確率変数

- この確率過程を、$N$個の状態変数を持つマルコフ連鎖で近似する

- そのために、まず、$x$のとる値についてグリッドを定める

$$
  x_{i} \in \mathcal{X} = \{x_{1},x_{2},...,x_{N}\}
$$

- 例えば、以下のような等間隔のグリッドを考える

\begin{align*}
  x_{1} &= \frac{-m\sigma_{\varepsilon}}{\sqrt{1-\rho^{2}}} \\
  x_{N} &= \frac{m\sigma_{\varepsilon}}{\sqrt{1-\rho^{2}}} \\
  x_{i} &= x_{i-1} + w \text{ for }i=2,...,N-1
\end{align*}

ここで、$w=\frac{x_{N}-x_{1}}{N-1}$

- 平均値ゼロから上下それぞれ標準偏差$\frac{\sigma_{\varepsilon}}{\sqrt{1-\rho^{2}}}$の$m$倍の範囲に等間隔にグリッドを定める

- グリッドを定めたら、次にグリッドの集合に含まれる一つの点から別の点に移る確率を求める

- ここで、$\varepsilon'$は正規分布に従うので、

$$
  \varepsilon' = x' - c - \rho x \sim N(0,\sigma_{\varepsilon}^{2})
$$

- すると、$x_{i}$を所与として、$x'$が$x_{j}$の近傍にある確率は、以下のように近似できる

\begin{align*}
  p_{i1} &= \Phi\left(x'+\frac{w}{2}-c-\rho x_{i}\right), \text{ for } x'=x_{1} \\
  p_{ij} &= \Phi\left(x'+\frac{w}{2}-c-\rho x_{i}\right) \\
  &-\Phi\left(x'-\frac{w}{2}-c-\rho x_{i}\right), \text{ for } x'=x_{j}, j=2,3,...,N-1 \\
  p_{iN} &= 1-\Phi\left(x'-\frac{w}{2}-c-\rho x_{i}\right), \text{ for } x'=x_{N}
\end{align*}

ここで、$\Phi(\cdot)$は正規分布$N(0,\sigma_{\varepsilon}^{2})$の累積分布関数である

$x'$がグリッドの端にある場合に注意

- 全ての$i,j=1,...,N$について確率$p_{ij}$を計算すると、以下の遷移行列が求まる

$$
  P = \left[\begin{array}{cccc}
  p_{11} & \cdots & \cdots & p_{1N} \\
  \vdots & \ddots &        & \vdots \\
  \vdots &        & \ddots & \vdots \\
  p_{N1} & \cdots & \cdots & p_{NN}
  \end{array}\right]
$$

In [1]:
%%file tauchen.m
function [Z,Zprob] = tauchen(N,mu,rho,sigma,m)

Z     = zeros(N,1); % グリッド
Zprob = zeros(N,N); % 遷移確率の行列
c     = (1-rho)*mu; % 定数項

% 等間隔のグリッドを定める
% 最大値と最小値
zmax  = m*sqrt(sigma^2/(1-rho^2));
zmin  = -zmax;
% グリッド間の間隔
w = (zmax-zmin)/(N-1);

Z = linspace(zmin,zmax,N)';
% 定常状態はmu
Z = Z + mu;

% グリッドを所与として、遷移確率を求める
for j = 1:N
    for k = 1:N
        if k == 1
            Zprob(j,k) = cdf_normal((Z(1)-c-rho*Z(j)+w/2)/sigma);
        elseif k == N
            Zprob(j,k) = 1 - cdf_normal((Z(N)-c-rho*Z(j)-w/2)/sigma);
        else
            Zprob(j,k) = cdf_normal((Z(k)-c-rho*Z(j)+w/2)/sigma) - ...
                         cdf_normal((Z(k)-c-rho*Z(j)-w/2)/sigma);
        end
    end
end

% 正規分布の累積分布関数
function c = cdf_normal(x)
    c = 0.5*erfc(-x/sqrt(2));

Created file 'C:\Users\takek\Documents\macrotopicsA2020\tauchen.m'.


- $p_{ij}$の計算についてのグラフ

- $x'=\rho x_{i}$のとき、$\varepsilon' = x' - \rho x = 0$になり、確率密度関数$\phi(\varepsilon')=\phi(x' - \rho x_{i})=\phi(0)$は最大値をとる

- ここで、$x' = x_{j}$のときの確率密度関数$\phi(x_{j} - \rho x_{i})$は、$\Phi\left(x'+\frac{w}{2}-c-\rho x_{i}\right)-\Phi\left(x'-\frac{w}{2}-c-\rho x_{i}\right)$で近似できる

## 準線形モデルにおける最適金融政策

- 時間反復法を使った実際の応用例として、準線形モデルを用いて、ゼロ金利制約下での最適金融政策を考えたAdam and Billi(2006; 2007)を紹介する

- ニューケインジアン・モデルにおいては、価格の硬直性により相対価格に歪みがあるため、一般には、社会計画者問題における最適配分を実現できない

- 最適金融政策とは、社会計画者問題における配分にできるだけ近い配分を達成するような金融政策のことである
  - 代表的家計の価値関数を2次関数で近似した目的関数を社会厚生とみなして、線形近似した均衡条件を制約として最大化するような配分を求める

- 動学最適化問題において、政策決定者が現在だけでなく将来の変数についても操作し、将来の経路にコミットできるとき、これを最適コミットメント政策という

- そのようなコミットができない場合は、政策決定者は毎期ごとの社会厚生を最大化するように変数を選ぶことになり、これを最適裁量政策という

- 政策決定者は、以下の社会厚生を最大化するように$\{y_{t},\pi_{t},r_{n,t}\}_{t=0}^{\infty}$の流列を求める

$$
  \max_{\{y_{t},\pi_{t},r_{n,t}\}_{t=0}^{\infty}} -\mathbb{E}_{0}\sum_{t=0}^{\infty}\beta^{t}\left(\pi_{t}^{2} + \lambda y_{t}^{2}\right)
$$

subject to

\begin{align*}
  y_{t} &= \mathbb{E}_{t}y_{t+1} - \left(r_{n,t}-\mathbb{E}_{t}\pi_{t+1}\right) + g_{t} \\
  \pi_{t} &= \kappa y_{t} + \beta \mathbb{E}_{t}\pi_{t+1} +u_{t} \\
  r_{n,t} &\geq 0
\end{align*}

- 社会厚生は、現在から無限先の将来にわたっての、産出ギャップとインフレ率の2乗の加重和の割引現在価値で与えられる

- 政策決定者は、名目金利だけでなくその他の変数も直接選ぶことに注意

- $(g_{t},u_{t})$は外生的なショックであり、それぞれ以下のAR(1)過程に従う

\begin{align*}
  g_{t+1} = (1-\rho_{g})g + \rho_{g}g_{t} + \varepsilon_{g,t+1}, &\quad \varepsilon_{g,t+1} \sim N(0,\sigma_{g}^{2}) \\ 
  u_{t+1} = \rho_{u}u_{t} + \varepsilon_{u,t+1}, &\quad \varepsilon_{u,t+1} \sim N(0,\sigma_{u}^{2})
\end{align*}

$g_{t}$は需要面に対するショック（財政支出など）、$u_{t}$は供給面に対するショック（価格マークアップなど）と考えることができる

- またこの場合、$g_{t}$は定常状態で自然利子率の定常値に一致する($g=r^{*}$)

- 政策決定者の動学最適化問題は、前回でも出てきた動的ラグランジュアンとその1階の必要条件を用いて解くことができる

\begin{align*}
  \mathcal{L}_{0} \equiv & \mathbb{E}_{0}\sum_{t=0}^{\infty}\{\beta^{t}\left(\pi_{t}^{2} + \lambda y_{t}^{2}\right) \\
  & + 2\phi_{PC,t}\left(-\pi_{t} + \kappa y_{t} + \beta \mathbb{E}_{t}\pi_{t+1} +u_{t}\right) \\
  & + 2\phi_{EE,t}\left(-y_{t} + \mathbb{E}_{t}y_{t+1} - \left(r_{n,t}-\mathbb{E}_{t}\pi_{t+1}\right) + g_{t}\right) \\
  & + 2\phi_{ZLB,t}r_{n,t}\}
\end{align*}

ここで、$\{\phi_{PC,t},\phi_{EE,t},\phi_{ZLB,t}\}_{t=0}^{\infty}$はそれぞれの均衡条件にかかるラグランジュ乗数である

- 最適コミットメント政策では、政策決定者は、時点0で現在から将来にわたる変数について社会厚生を最大化するように選ぶ

- このとき、動的ラグランジュアンの1階の必要条件は

\begin{align*}
  \pi_{t}: & \pi_{t} - \phi_{PC,t} + \phi_{PC,t-1} + \beta^{-1}\phi_{EE,t-1} = 0 \\ 
  y_{t}: & \lambda y_{t} + \kappa \phi_{PC,t} - \phi_{EE,t} + \beta^{-1}\phi_{EE,t-1} = 0 \\ 
  r_{n,t}: & -\phi_{EE,t} + \phi_{ZLB,t} = 0 
\end{align*}

となる

- ここで、$\phi_{ZLB,t}$はゼロ金利制約にかかるラグランジュ乗数であり、ゼロ金利制約がバインドしない限りはゼロになる

- このことは以下の相補スラック条件にまとめられる

$$
  \phi_{ZLB,t} r_{n,t} = 0, \quad \phi_{ZLB,t} \geq 0, \quad r_{n,t} \geq 0
$$

- すなわち、$\phi_{ZLB,t} > 0$あるいは$r_{n,t} > 0$のいずれかが成り立ち、均衡条件は$r_{n,t}$の値によって場合分けされる

- 最適裁量政策では、政策決定者は将来の変数にコミットできず、毎期ごとにその時点の変数を社会厚生を最大化するように選ぶ

- この場合、$(\mathbb{E}_{t}y_{t+1},\mathbb{E}_{t}\pi_{t+1})$を所与とした動的ラグランジュアンの1階条件から、解となる政策関数を求めることができる。すなわち、

\begin{align*}
  \pi_{t}: & \pi_{t} - \phi_{PC,t} = 0 \\ 
  y_{t}: & \lambda y_{t} + \kappa \phi_{PC,t} - \phi_{EE,t} = 0 \\ 
  r_{n,t}: & -\phi_{EE,t} + \phi_{ZLB,t} = 0 
\end{align*}


- 相補スラック条件より、もし$r_{n,t} > 0$であれば、$\phi_{ZLB,t} = 0$である。このとき均衡条件は、

\begin{align*}
  r_{n,t} &= -y_{t} + \mathbb{E}_{t}y_{t+1} + \mathbb{E}_{t}\pi_{t+1} + g_{t} \\
  \pi_{t} &= \kappa y_{t} + \beta \mathbb{E}_{t}\pi_{t+1} +u_{t} = 0 \\
  0 &= \lambda y_{t} + \kappa \pi_{t}
\end{align*}

- あるいは、もし$r_{n,t}=0$であれば、$\phi_{ZLB,t} = \phi_{EE,t} > 0$である。このとき均衡条件は、

\begin{align*}
  0 &= -y_{t} + \mathbb{E}_{t}y_{t+1} + \mathbb{E}_{t}\pi_{t+1} + g_{t} \\
  \pi_{t} &= \kappa y_{t} + \beta \mathbb{E}_{t}\pi_{t+1} +u_{t} = 0 \\
  \phi_{EE,t} &= \lambda y_{t} + \kappa \pi_{t}
\end{align*}

- いずれの場合も、3つの未知数に対して3つの式があるので、解を求めることができる

## 時間反復法による最適裁量政策の数値計算

- ここでは、時間反復法を用いてAdam and Billi(2007)の結果を再現してみよう

- 最適裁量政策においては、2状態モデルと同じように、モデルの状態変数が外生的なショック以外に存在しないため、外生変数をグリッドとしてそれぞれのグリッドの点で均衡条件を解けばよい

- まず、外生的なショックの確率過程をTauchen(1986)の方法を用いてマルコフ連鎖で近似する

- 各変数のグリッドは以下で与えられる

\begin{align*}
  g \in \{g_{1},g_{2},...,g_{N_{g}}\} \\
  u \in \{u_{1},u_{2},...,u_{N_{u}}\}
\end{align*}



- また、遷移確率は、それぞれ、

$$
  P^{g} = \left[\begin{array}{cccc}
  p_{11}^{g} & \cdots & \cdots & p_{1N}^{g} \\
  \vdots & \ddots &        & \vdots \\
  \vdots &        & \ddots & \vdots \\
  p_{N1}^{g} & \cdots & \cdots & p_{NN}^{g}
  \end{array}\right]
  , \quad
  P^{u} = \left[\begin{array}{cccc}
  p_{11}^{u} & \cdots & \cdots & p_{1N}^{u} \\
  \vdots & \ddots &        & \vdots \\
  \vdots &        & \ddots & \vdots \\
  p_{N1}^{u} & \cdots & \cdots & p_{NN}^{u}
  \end{array}\right]
$$

- これらの2つのマルコフ連鎖を結合して1つのマルコフ連鎖として考えるために、2次元のグリッドの座標$(g,u)$を1次元のベクトル$s$に集約する

- 例えば、$N_{g}=N_{u}=2$の場合、$g$と$u$の組み合わせは以下の$N_{g} \times N_{u}=4$個のベクトルで与えられる

\begin{align*}
  s_{1} &= (g_{1},u_{1}) \\ 
  s_{2} &= (g_{1},u_{2}) \\ 
  s_{3} &= (g_{2},u_{1}) \\ 
  s_{4} &= (g_{2},u_{2})
\end{align*}

- ここで、ベクトル$s$のインデックス$i \in \{1,2,3,4\}$は、それぞれのショックのインデックスの組に対応しており、

$$
  s_{i} = (g_{k(i)},u_{l(i)})
$$

のように書ける

- また、それぞれの遷移行列のクロネッカー積、$P=P^{g} \otimes P^{u}$が、結合されたマルコフ連鎖の遷移行列となる

- 例えば、$N_{g}=N_{u}=2$の場合、

$$
  P = \left[\begin{array}{cccc}
  p_{11}^{g}p_{11}^{u} & p_{11}^{g}p_{11}^{u} & p_{12}^{g}p_{11}^{u} & p_{12}^{g}p_{12}^{u} \\
  p_{11}^{g}p_{21}^{u} & p_{11}^{g}p_{22}^{u} & p_{12}^{g}p_{21}^{u} & p_{12}^{g}p_{22}^{u} \\
  p_{21}^{g}p_{11}^{u} & p_{21}^{g}p_{11}^{u} & p_{22}^{g}p_{11}^{u} & p_{22}^{g}p_{12}^{u} \\
  p_{21}^{g}p_{21}^{u} & p_{21}^{g}p_{22}^{u} & p_{22}^{g}p_{21}^{u} & p_{22}^{g}p_{22}^{u} \\
  \end{array}\right]
$$

- 数値計算のアルゴリズムは以前の2状態モデルとほぼ同じである

- アルゴリズム（再掲）


1. グリッド生成：状態空間の評価点を有限個のグリッドに区切る。この場合、$(s_{1},...,s_{N_{g}N_{u}})$はすでに与えられている

2. 収束の基準：収束の基準になるパラメータ$\varepsilon$を与える

3. 最適化：古い政策関数$y = \varsigma_{y}^{(n-1)}(s),\pi = \varsigma_{\pi}^{(n-1)}(s),r_{n} = \varsigma_{r_{n}}^{(n-1)}(s)$を所与として、各$s_{i}$について、以下の3本の式を$(y_{i},\pi_{i},r_{n,i})$について解く

\begin{align*}
  r_{n,i} &= -y_{i} + y_{i}^{e} + \pi_{i}^{e} + g_{k(i)} \\
  \pi_{i} &= \kappa y_{i} + \beta \pi_{i}^{e} + u_{l(i)} \\
  0 &= \lambda y_{i} + \kappa \pi_{i}
\end{align*}

ここで、

\begin{align*}
  y_{i}^{e} &= \sum_{j=1}^{N}p_{ij}\varsigma_{y}^{(n-1)}(s_{j}) \\
  \pi_{i}^{e} &= \sum_{j=1}^{N}p_{ij}\varsigma_{\pi}^{(n-1)}(s_{j})
\end{align*}

である

$r_{n,i} > 0$が満たされているかをチェックし、そうでなければ、$r_{n,i}=0$として、以下の2本の式を$(y_{i},\pi_{i})$について解く

\begin{align*}
  0 &= -y_{i} + y_{i}^{e} + \pi_{i}^{e} + g_{k(i)} \\
  \pi_{i} &= \kappa y_{i} + \beta \pi_{i}^{e} + u_{l(i)} \\
\end{align*}

このステップで、新しい政策関数のベクトル$\varsigma^{(n)}(s_{i})$を得る

4. 全ての$s_{i}$について$\|\varsigma^{(n)}(s_{i})-\varsigma^{(n-1)}(s_{i})\|<\varepsilon$であればストップ。そうでなければ、$\varsigma^{(n)}(s_{i})$を$\varsigma^{(n-1)}(s_{i})$に代入して、ステップ3-4を繰り返す

# 非線形モデル

- これまでは均衡条件を対数線形近似したモデルでゼロ金利制約による非線形性を考慮してきた

- しかし、元の均衡条件は非線形であり、そのような非線形の均衡条件とゼロ金利制約を合わせて考えることももちろん可能である

- ここでは、最終財生産企業と中間財生産企業、家計、政府からなるニューケインジアンモデルを考える
  - 価格の硬直性について、Rotemberg(1982)型のインフレ率の定常状態からの乖離の2乗に比例する調整費用を考える

## 最終財生産企業

- 最終財生産企業は、異なる中間財$Y_{t}(j), j \in [0,1]$を用いて、最終財を生産する


$$
  Y_{t} = \left( \int_{0}^{1} Y_{t}(j)^{1-\nu}dj \right)^{\frac{1}{1-\nu}}
$$

ここで、$\nu$は需要の価格弾力性の逆数である

- 利潤最大化あるいは費用最小化により、以下の需要曲線を得る

$$
  Y_{t}(j) = \left(\frac{P_{t}(j)}{P_{t}}\right)^{-1/\nu} Y_{t}
$$

$P_{t}(j)$は中間投入$Y_{t}(j)$の価格である。また、一般価格水準は$P_{t}=\left( \int_{0}^{1} P_{t}(j)^{\frac{\nu-1}{\nu}}dj \right)^{\frac{\nu}{\nu-1}}$と表される

## 中間財生産企業

- それぞれの中間財生産企業は、$Y_{t}(j)$を独占競争下で生産し、2次の価格調整コストの下で価格を設定する

- 生産関数は以下で与えられる
$$
  Y_{t}(j) = A_{t}N_{t}(j)
$$
  - $A_{t}$: 全要素生産性
  - $N_{t}(j)$: 労働投入

- $A_{t}$の成長率は、

$$
  \gamma_{t} \equiv \ln(A_{t}/A_{t-1})=\gamma+z_{t}
$$

のように確定的なトレンド$\gamma$とトレンドに対するショック$z_{t}$に分解できる

- $z_{t}$は、AR(1)過程$z_{t} = \rho_{z} z_{t-1} + \varepsilon_{z,t}, \varepsilon_{z,t} \sim N(0,\sigma_{z}^{2})$に従う
 

- 中間財$j$を生産する企業は、その企業自身の生産関数と、最終財生産企業からの需要関数を制約条件として、以下の将来の期待利益の割引現在価値を最大化するように中間財価格$P_{t}(j)$を選ぶ
$$
  \mathbb{E}_{t} \left[ \sum_{s=0}^{\infty} \beta^{s}Q_{t+s|t} \left( \frac{P_{t+s}(j)}{P_{t+s}}Y_{t+s}(j)-W_{t+s}N_{t+s}(j)-AC_{t+s}(j) \right) \right]
$$
ここで、$\beta^{s}Q_{t+s|t}=\beta^{s}(C_{t+s}/C_{t})^{-\tau}(A_{t}/A_{t+s})^{1-\tau}$は確率的割引因子と呼ばれ、家計のオイラー方程式から導出される
  - $\beta$: 主観的割引因子
  - $\tau$: リスク回避度に関するパラメータ
  - $C_{t}$: 総消費

- 価格の調整費用は、Rotemberg(1982)に従い、
$$
  AC_{t}(j) = \frac{\phi}{2}\left( \frac{P_{t}(j)}{P_{t-1}(j)}-\Pi \right)^{2}Y_{t}(j)
$$
として定式化される
  - $\phi$: 調整費用の大きさに関するパラメータ
  - $\Pi$: 定常状態での粗インフレ率
  - $W_{t}$: 実質賃金率

- $P_{t}(j)$について微分し整理することで、最適化の必要条件は以下の式で与えられる

\begin{align*}
 & (1-\nu^{-1})\left(\frac{P_{t}(j)}{P_{t}}\right)^{-1/\nu}\frac{Y_{t}}{P_{t}}+\nu^{-1}\frac{W_{t}}{A_{t}}\left(\frac{P_{t}(j)}{P_{t}}\right)^{-1/\nu-1}\frac{Y_{t}}{P_{t}} \\
 & -\frac{\partial AC_{t}(j)}{\partial P_{t}(j)}-\beta\mathbb{E}_{t}Q_{t+1|t}\frac{\partial AC_{t+1}(j)}{\partial P_{t}(j)}=0
\end{align*}

## 家計

- 代表的家計は、以下の効用最大化問題を解く
$$
  \mathbb{E}_{t} \left[ \sum_{s=0}^{\infty} \beta^{s}\left( \frac{(C_{t+s}/A_{t+s})^{1-\tau}-1}{1-\tau} - \chi_{H}H_{t+s} \right) \right]
$$
subject to
$$
  P_{t}C_{t} + B_{t} + T_{t} = P_{t}W_{t}H_{t} + R_{n,t-1}B_{t-1} + P_{t}D_{t} + P_{t}SC_{t}
$$
  - $\chi_{H}$: 労働による不効用の効用全体に対するウェイト
  - $B_{t}$: $t$期末における債券の保有量、
  - $H_{t}$: 総労働供給、
  - $T_{t}$: 所得移転、
  - $R_{n,t}$: 名目金利を\textbf{粗利(gross)}で表したもの
  - $D_{t}$: 家計が保有する中間財生産企業からの配当
  - $SC_{t}$: 状態依存型証券を取引することで得られる純キャッシュフロー


- 最適化の必要条件は以下の式で与えられる

\begin{align*}
 & 1=\beta\mathbb{E}_{t}\left[\left(\frac{C_{t+1}/C_{t}}{A_{t+1}/A_{t}}\right)^{-\tau}\frac{A_{t}}{A_{t+1}}\frac{R_{n,t}}{\Pi_{t+1}}\right] \\
 & \frac{W_{t}}{A_{t}}=\chi_{H}\left(\frac{C_{t}}{A_{t}}\right)^{\tau}
\end{align*}
ここで、$\Pi_{t}=P_{t}/P_{t-1}$は粗インフレ率

## モデルを閉じる

- 金融政策ルールは以下の形をとる
$$
  R_{n,t} = \left( R\Pi^{*}\left(\frac{\Pi_{t}}{\Pi^{*}}\right)^{\psi_{1}}\left(\frac{Y_{t}}{Y_{t}^{*}}\right)^{\psi_{2}} \right)^{1-\rho_{R}} R_{n,t-1}^{\rho_{R}} \exp(\varepsilon_{R,t})
$$
  - $R$: 定常状態の実質利子率
  - $\Pi^{*}$: 目標インフレ率
  - $\varepsilon_{R,t} \sim N(0,\sigma_{R}^{2})$: 金融政策ショック 
  - $Y_{t}^{*}=(1-\nu)^{1/\tau}A_{t}\exp(g_{t})$: 自然産出量（$\phi=0$のときの$Y_{t}$の水準）

- 財政支出は、$G_{t}=(1-\exp(-g_{t}))Y_{t}$で与えられる。ここで、

$$
  g_{t} = (1-\rho_{g})g + \rho_{g}g_{t-1} + \varepsilon_{g,t}, \quad \varepsilon_{g,t} \sim N(0,\sigma_{g}^{2})
$$

- 全ての中間財生産企業が同じ決定をする対象均衡に着目する（添字$j$は省略）

- 市場清算条件は以下で与えられる

\begin{align*}
  C_{t} + G_{t} + AC_{t} &= Y_{t} \\
  H_{t} &= N_{t}
\end{align*}

- $(Y_{t},Y_{t}^{*},C_{t})$は共通のトレンド$A_{t}$を持つことから、$\tilde{Y}_{t}=Y_{t}/A_{t}, \tilde{Y}_{t}^{*}=Y_{t}^{*}/A_{t}, \tilde{C}_{t}=C_{t}/A_{t}$のようにトレンドを除去する

- 結果として、均衡条件は以下で与えられる

\begin{align*}
 & \tilde{C}_{t}^{-\tau}=\beta R_{n,t}\mathbb{E}_{t}\left[\frac{\tilde{C}_{t+1}^{-\tau}}{\exp(\gamma_{t+1})\Pi_{t+1}}\right] \tag{4} \\
 & 0=\left((1-\nu^{-1})+\nu^{-1}\tilde{C}_{t}^{\tau}-\phi\left(\Pi_{t}-\bar{\Pi}\right)\left[\Pi_{t}-\frac{1}{2\nu}\left(\Pi_{t}-\bar{\Pi}\right)\right]\right)\tilde{C}_{t}^{-\tau}\tilde{Y}_{t} \\
 & +\beta\phi\mathbb{E}_{t}\left[\tilde{C}_{t+1}^{-\tau}\tilde{Y}_{t+1}\left(\Pi_{t+1}-\bar{\Pi}\right)\Pi_{t+1}\right] \tag{5} \\
 & \tilde{C}_{t}+\frac{\phi}{2}\left(\Pi_{t}-\Pi\right)^{2}\tilde{Y}_{t}=\exp(-g_{t})\tilde{Y}_{t} \tag{6} \\
 & R_{n,t}^{*}=\left(R^{*}\bar{\Pi}\left(\frac{\Pi}{\bar{\Pi}}\right)^{\psi_{1}}\left(\frac{\tilde{Y}_{t}}{\tilde{Y}_{t}^{*}}\right)^{\psi_{2}}\right)^{1-\rho_{R}}R_{n,t-1}^{*\rho_{R}}\exp(\varepsilon_{R,t}) \tag{7} \\
 & R_{n,t}=\max\left\{ R_{n,t}^{*},1\right\} \tag{8}
\end{align*}
これらの4つの式は、消費のオイラー方程式、ニューケインジアン・フィリップス曲線、経済の資源制約、そしてテイラー型の金融政策ルールである。
$R_{n,t}^{*}$はシャドーレートである。$R_{n,t}$はゼロ金利制約から1以上となる(純利に直すと0以上となる)。


- これらの非線形の均衡条件にも、時間反復法を適用できる

- 前節までの準線形モデルと比べて、ゼロ金利制約以外の非線形性を考慮した場合、モデルの政策関数は特に定常状態から大きく離れたところでは異なり得る

- また、非線形の均衡条件を対数線形近似することで、(1)-(3)式に近い式を求めることができる